아이템 기반 필터링 구현하기(작업자 : 권준호)

- 사용자 선호에 따른 장르, 감독별, 줄거리 유사도 검증 구현 

In [56]:
##########################################

# 비회원 장르별 순위 추천


In [57]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select m.*, g.*, d.*, a.*
    from movie m
    left outer join moviegenre mg
    on m.moviecd = mg.moviecd
    left outer join genre g
    on mg.genrecd = g.genrecd
    left outer join moviedirector md
    on m.moviecd = md.moviecd
    left outer join director d
    on md.directorcd = d.directorcd
    left outer join movieactor ma
    on m.moviecd = ma.moviecd
    left outer join actor a
    on ma.actorcd = a.actorcd
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_content = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd', 'genreCd', 'genreNm', 'directorCd', 'directorNm', 'directorNmEn', 'actorCd', 'actorNm', 'actorNmEn'])
movie_content

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreCd,genreNm,directorCd,directorNm,directorNmEn,actorCd,actorNm,actorNmEn
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,1408,헨리 월설,None
1,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,7525,랄프 루이스,None
2,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,13797,릴리안 기쉬,None
3,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,1408,헨리 월설,None
4,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,7525,랄프 루이스,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520821,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,7,코미디,40137,최기윤,None,1,,None
520822,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,16672,최기윤,None,1,,None
520823,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,40137,최기윤,None,1,,None
520824,20219995,이멀전씨 키트,Emergency Kit,None,2021,None,0,None,None,None,4,기타,16747,전건우,None,1,,None


In [58]:
# 2. 데이터 전처리하기
# 2-1. shortContent, genreNm, directNm별로 전처리

movie_content1 = movie_content[['movieCd', 'movieNm', 'actorNm']]
movie_content2 = movie_content[['movieCd', 'movieNm', 'genreNm']]
movie_content3 = movie_content[['movieCd', 'movieNm', 'directorNm']]

#2-2. 결측치 ""로 대체하기
movie_content1["actorNm"] = movie_content1['actorNm'].fillna("")
movie_content2["genreNm"] = movie_content2['genreNm'].fillna("")
movie_content3["directorNm"] = movie_content3['directorNm'].fillna("")

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [59]:
# 2-3. shortContent 
import numpy as np

movie_content1["actorNm"] = movie_content1.groupby(by=["movieCd", "movieNm"])["actorNm"].transform(lambda row: '$$'.join(row))
movie_content1.drop_duplicates(inplace=True)
movie_content1['actorNm'] = movie_content1['actorNm'].map(lambda sc : ','.join(np.unique(sc.split('$$'))))
movie_content1.shape

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

(46918, 3)

In [60]:
movie_content2["genreNm"] = movie_content2.groupby(by=["movieCd", "movieNm"])["genreNm"].transform(lambda row: ','.join(row))
movie_content2.drop_duplicates(inplace=True)
movie_content2['genreNm'] = movie_content2['genreNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content2.shape

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(46918, 3)

In [61]:
movie_content3["directorNm"] = movie_content3.groupby(by=["movieCd", "movieNm"])["directorNm"].transform(lambda row: ','.join(row))
movie_content3.drop_duplicates(inplace=True)
movie_content3['directorNm'] = movie_content3['directorNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content3.shape

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(46918, 3)

In [62]:
# 2-4. movieCd, movieNm 기준으로 통합하기
movie_contents_merge = movie_content1.merge(movie_content2, on=['movieCd', 'movieNm'], how="inner")\
                                        .merge(movie_content3, on=['movieCd', 'movieNm'], how="inner")

In [63]:
print( movie_contents_merge.shape )
movie_contents_merge.head()

(46918, 5)


,movieCd,movieNm,actorNm,genreNm,directorNm
0,19158001,국가의 탄생,"랄프 루이스,릴리안 기쉬,헨리 월설","드라마,서부극(웨스턴),전쟁",D.W. 그리피스
1,19188001,개의 일생,찰리 채플린,코미디,찰리 채플린
2,19190002,여름밤의 기묘한 이야기,"아니타 베르버,콘라드 바이트",공포(호러),리차드 오스왈드
3,19190003,재판장,"리차드 크리스텐슨,할바드 호프",드라마,칼 테오도르 드레이어
4,19190007,의리적 구토,"김도산,윤혁,이경환","드라마,사극,액션",김도산


In [64]:
# 2-5. content 컬럼 생성 
# shortContent + genreNm + directorNm 통합

movie_contents_merge['content'] = movie_contents_merge["actorNm"] + movie_contents_merge["genreNm"] + movie_contents_merge["directorNm"]
movie_contents_merge.head()

,movieCd,movieNm,actorNm,genreNm,directorNm,content
0,19158001,국가의 탄생,"랄프 루이스,릴리안 기쉬,헨리 월설","드라마,서부극(웨스턴),전쟁",D.W. 그리피스,"랄프 루이스,릴리안 기쉬,헨리 월설드라마,서부극(웨스턴),전쟁D.W. 그리피스"
1,19188001,개의 일생,찰리 채플린,코미디,찰리 채플린,찰리 채플린코미디찰리 채플린
2,19190002,여름밤의 기묘한 이야기,"아니타 베르버,콘라드 바이트",공포(호러),리차드 오스왈드,"아니타 베르버,콘라드 바이트공포(호러)리차드 오스왈드"
3,19190003,재판장,"리차드 크리스텐슨,할바드 호프",드라마,칼 테오도르 드레이어,"리차드 크리스텐슨,할바드 호프드라마칼 테오도르 드레이어"
4,19190007,의리적 구토,"김도산,윤혁,이경환","드라마,사극,액션",김도산,"김도산,윤혁,이경환드라마,사극,액션김도산"


In [65]:
# 컬럼별 ','으로 구분하여 통합시키기
colums = ["actorNm", "genreNm", "directorNm"]
movie_contents_merge['content'] = movie_contents_merge[colums].apply(lambda row: ','.join(row.values), axis=1)
movie_contents_merge

,movieCd,movieNm,actorNm,genreNm,directorNm,content
0,19158001,국가의 탄생,"랄프 루이스,릴리안 기쉬,헨리 월설","드라마,서부극(웨스턴),전쟁",D.W. 그리피스,"랄프 루이스,릴리안 기쉬,헨리 월설,드라마,서부극(웨스턴),전쟁,D.W. 그리피스"
1,19188001,개의 일생,찰리 채플린,코미디,찰리 채플린,"찰리 채플린,코미디,찰리 채플린"
2,19190002,여름밤의 기묘한 이야기,"아니타 베르버,콘라드 바이트",공포(호러),리차드 오스왈드,"아니타 베르버,콘라드 바이트,공포(호러),리차드 오스왈드"
3,19190003,재판장,"리차드 크리스텐슨,할바드 호프",드라마,칼 테오도르 드레이어,"리차드 크리스텐슨,할바드 호프,드라마,칼 테오도르 드레이어"
4,19190007,의리적 구토,"김도산,윤혁,이경환","드라마,사극,액션",김도산,"김도산,윤혁,이경환,드라마,사극,액션,김도산"
...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,"김지경,주수정",드라마,이수학,"김지경,주수정,드라마,이수학"
46914,20219978,볼레로 만들기,,기타,이와,",기타,이와"
46915,20219984,어디에도 없는 시간,,드라마,장태구,",드라마,장태구"
46916,20219993,청수탕 사람들,,"드라마,코미디",최기윤,",드라마,코미디,최기윤"


In [73]:
movie_contents_merge = movie_contents_merge.iloc[0:30000]

In [66]:
######################################################################

In [74]:
# 4. 문자열 genreNm 칼럼을 Count 기반으로 피처 벡터화 변환
# CountVectorizer = 단어 벡터화 카운트 정렬
# TfidfTransformer = TF-IDF로 단어 벡터화

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,1))
merge_content = count_vect.fit_transform(movie_contents_merge['content'])

tfidf_transformer = TfidfTransformer()

merge_content = tfidf_transformer.fit_transform(merge_content)

print(merge_content.shape)

(30000, 38776)


In [75]:
# 5. 리스트 객체 문자열 변경, count 피처 벡터화하기

from sklearn.metrics.pairwise import cosine_similarity

director_content_sim = cosine_similarity(merge_content, merge_content)
director_content_sim[0]


array([1.        , 0.        , 0.        , ..., 0.01212664, 0.        ,
       0.        ])

In [78]:
# 6. 특정영화 정보, 유사도 뽑아내기

def find_genre_score(movie_contents_merge, sim_metrics, title_name, top_n=100):

    # 입력한 영화 인덱스
    movieNm = movie_contents_merge[movie_contents_merge['movieNm'] == title_name]
    movieNm_index = movieNm.index.values

    # 입력한 영화 유사도 데이터 프레임을 추가하기
    movie_contents_merge["similarity"] = sim_metrics[movieNm_index, :].reshape(-1,1)

    # 유사도 내림차순 정렬(상위 인덱스 100개 추출)
    temp = movie_contents_merge.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[ : top_n]

    return movie_contents_merge.iloc[final_index]

In [80]:
# 7. 특정영화 유사한 장르 영화를 평점 순위별로 추천하기

# 7-1. 해리포터와 높은 유사도를 가지는 영화 순위별 정렬하기
similar_movie = find_genre_score(movie_contents_merge, director_content_sim, '해리포터와 비밀의 방', 30)
similar_movie = similar_movie[['movieCd', 'movieNm', 'similarity', 'content']]
similar_movie





C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,movieCd,movieNm,similarity,content
10823,20020222,해리포터와 비밀의 방,1.000000,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,드라마,크리스 콜럼버스"
9823,20010291,해리포터와 마법사의 돌,0.957997,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,가족,판타지,크리스 콜럼버스"
19494,20101224,해리 포터와 죽음의 성물1,0.730957,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,미스터리,판타지,데이빗 예이츠"
20203,20111009,해리포터와 죽음의 성물2,0.730957,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,미스터리,판타지,데이빗 예이츠"
18299,20090379,해리 포터와 혼혈 왕자,0.726426,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,가족,액션,어드벤처,데이빗 예이츠"
12563,20040598,해리포터와 아즈카반의 죄수,0.724023,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,드라마,알폰소 쿠아론"
13787,20050310,해리 포터와 불의 잔,0.608842,"다니엘 래드클리프,랄프 파인즈,로버트 패틴슨,루퍼트 그린트,엠마 왓슨,액션,판타지,..."
15780,20070063,해리 포터와 불사조 기사단,0.566618,"다니엘 래드클리프,루퍼트 그린트,액션,어드벤처,판타지,데이빗 예이츠"
8558,20000087,바이센테니얼 맨,0.345354,"로빈 윌리엄스,샘 닐,SF,드라마,멜로/로맨스,크리스 콜럼버스"
4212,19910355,나홀로 집에,0.298787,"다니엘 스톤,맥컬리 컬킨,조 페시,가족,스릴러,어드벤처,코미디,크리스 콜럼버스"
